# Mean Decrease impurity
* this notebook will use sklearn random forest mean decrease impurity to select the most important features.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import SPoC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as tts
import mne

from sklearn.model_selection import StratifiedKFold
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator,
                          cross_val_multiscore, LinearModel, get_coef)
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE 
import os
from mne.decoding import UnsupervisedSpatialFilter
import os.path as op
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from scipy.stats import skew, kurtosis
import mne 

from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.svm import SVC  # noqa
from sklearn.model_selection import ShuffleSplit  # noqa

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from sklearn.metrics import confusion_matrix
from mne.viz import tight_layout

import time
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
%matplotlib inline

import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from urllib.request import urlopen

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)

In [3]:
%run Batch_ArtifactFilter_Epoch.ipynb

In [3]:
epoch=mne.read_epochs('D:\\USB Drive\\NewEEG-200s\\New\\20131216_1310_epo.fif')

Reading D:\USB Drive\NewEEG-200s\New\20131216_1310_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1759 matching events found
Applying baseline correction (mode: mean)
1759 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [4]:
## Drop bad Channels
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);

In [5]:
## convert 3d data to 2d
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1759, 32125) (1759, 125, 257)


In [6]:
### Methods for convert our events into binary, 4: Language, 5:Non Language
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=1
        else:
            for i in range(len(events)):
                    events[i]=0
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [7]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

[1 1 1 ... 1 1 1]


In [8]:
## Logistical Regression
clf = make_pipeline(Vectorizer(),
                   StandardScaler(),
                         LogisticRegression())
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)

C:\Users\kingsly\Anaconda3\envs\mne\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
print(score)

0.5587121212121212


In [10]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 0.9159090909090909
0.6667614586818333


In [12]:
import eli5

In [13]:
eli5.explain_weights(clf) ## Find the most important features

Weight,Feature
0.6669 ± 0.0664,x31868
0.0073 ± 0.0336,x22209
0.0056 ± 0.0173,x17645
0.0047 ± 0.0152,x18628
0.0042 ± 0.0191,x22123
0.0042 ± 0.0170,x31452
0.0042 ± 0.0169,x4015
0.0036 ± 0.0216,x23115
0.0034 ± 0.0138,x25583
0.0034 ± 0.0201,x3275


In [24]:
eli5.show_prediction(clf, X[103])

In [34]:
channel=[]
index=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances)):
    if importances[i]>0.000001:
        channel.append(importances[i])
        index.append(i)
channel1=np.array(channel)
channel1.shape

(336,)

In [29]:
importances

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
clf.feature_importances_[]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.0002979 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00029079,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [35]:
index=np.array(index)
index

array([    7,    49,   293,   303,   516,   538,   707,   709,   793,
        1136,  1280,  1363,  1623,  1640,  1733,  1756,  1803,  1862,
        1876,  1906,  1948,  1998,  1999,  2011,  2145,  2531,  2624,
        2627,  2672,  2801,  2831,  2900,  2943,  3058,  3275,  3378,
        3430,  3503,  3641,  3664,  3679,  3728,  4009,  4015,  4022,
        4036,  4038,  4189,  4615,  4628,  5040,  5202,  5305,  5326,
        5364,  5406,  5483,  5493,  5641,  5716,  5763,  5795,  5848,
        5893,  5906,  5909,  6004,  6034,  6298,  6315,  6433,  6487,
        6503,  6531,  6537,  6751,  6759,  6762,  6777,  6778,  6917,
        7027,  7152,  7184,  7319,  7365,  7544,  7709,  7721,  7731,
        7989,  7994,  8285,  8423,  8555,  8566,  8599,  9270,  9381,
       10581, 10636, 10675, 10676, 10678, 10781, 10793, 10797, 10804,
       10908, 10941, 10954, 10956, 10968, 10995, 11006, 11016, 11052,
       11077, 11132, 11182, 11188, 11278, 11289, 11368, 11586, 11632,
       11948, 12367,

In [51]:
## Append the points with high features importants for our data
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
newdata=[]
j=0
for point in X:
    data=[]
    print(++j)
    for i in range(len(point)):
        if i in index:
            data.append(point[i])
    newdata.append(data)                   
        

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [52]:
newdata=np.array(newdata)
newdata.shape

(1759, 336)

In [54]:
## Ran random Forest on the selected data
X = newdata
y = epoch.events[:,-1]
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 0.9159090909090909
0.6677735028437604


In [55]:
eli5.explain_weights(clf)

Weight,Feature
0.6669 ± 0.0664,x335
0.0079 ± 0.0332,x209
0.0058 ± 0.0171,x161
0.0049 ± 0.0146,x177
0.0048 ± 0.0125,x116
0.0042 ± 0.0191,x205
0.0042 ± 0.0170,x327
0.0042 ± 0.0169,x43
0.0039 ± 0.0166,x186
0.0036 ± 0.0216,x224


In [57]:
clf.feature_importances_

array([0.00000000e+00, 0.00000000e+00, 1.88171576e-03, 0.00000000e+00,
       5.87857573e-04, 0.00000000e+00, 1.62969909e-03, 0.00000000e+00,
       1.28212962e-03, 2.00985942e-03, 2.81036783e-03, 0.00000000e+00,
       0.00000000e+00, 2.53626533e-04, 0.00000000e+00, 0.00000000e+00,
       9.03409431e-04, 3.02908142e-04, 5.93446006e-04, 5.83214179e-04,
       6.03816631e-04, 8.32551698e-04, 5.93649507e-04, 0.00000000e+00,
       2.43000382e-04, 2.68377336e-03, 1.75916925e-03, 1.10569653e-03,
       2.28951717e-04, 1.21642463e-03, 9.59672842e-04, 0.00000000e+00,
       0.00000000e+00, 6.06566277e-04, 3.35547903e-03, 6.00298745e-04,
       5.34629526e-04, 3.05897151e-04, 1.63947850e-03, 8.65883046e-04,
       3.13348771e-03, 1.15319265e-03, 0.00000000e+00, 4.17857466e-03,
       0.00000000e+00, 2.88567076e-03, 1.83658693e-03, 5.94029306e-04,
       3.04136452e-04, 5.99852378e-04, 0.00000000e+00, 1.60668653e-03,
       0.00000000e+00, 3.95839230e-04, 1.53048911e-04, 1.59829399e-03,
      

In [60]:
clf =LogisticRegression()
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)
    
clf.fit(X_train, y_train)
    
score = clf.score(X_test, y_test)
print(score)

0.6287878787878788


C:\Users\kingsly\Anaconda3\envs\mne\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [61]:
eli5.sklearn.explain_weights.explain_linear_classifier_weights(clf)

Explanation(estimator="LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class='warn',\n          n_jobs=None, penalty='l2', random_state=None, solver='warn',\n          tol=0.0001, verbose=0, warm_start=False)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1.4240277090452387, std=None, value=None), FeatureWeight(feature='x59', weight=1.3209353451147106e-05, std=None, value=None), FeatureWeight(feature='x171', weight=1.2018029353688385e-05, std=None, value=None), FeatureWeight(feature='x163', weight=1.0379213795322507e-05, std=None, value=None), FeatureWeight(feature='x68', weight=9.755164540692238e-06, std=None, value=None), FeatureWeight(feature='x309', weight=8.578184833479593e-06, std=None, value=None), FeatureWeight(feature='x169', weight=8.450286163857808e-06, std=None, value=None), FeatureWeight(feature='x69', weight=7.646178361836793e-06, std=None, value=None)], neg=[FeatureWeight(feature='x335', weight=-0.13298975543376212, std=None, value=None), FeatureWeight(feature='x110', weight=-1.2059058862897677e-05, std=None, value=None), FeatureWeight(feature='x6', weight=-1.1628030125637024e-05, std=None, value=None), FeatureWeight(feature='x7', weight=-9.032649517305785e-06, std=None, value=None), FeatureWeight(feature='x114', weight=-8.940534703431879e-06, std=None, value=None), FeatureWeight(feature='x306', weight=-8.899179041408397e-06, std=None, value=None), FeatureWeight(feature='x139', weight=-8.63346796966824e-06, std=None, value=None), FeatureWeight(feature='x111', weight=-8.291516754871603e-06, std=None, value=None), FeatureWeight(feature='x60', weight=-8.191094252628394e-06, std=None, value=None), FeatureWeight(feature='x75', weight=-7.745825589766683e-06, std=None, value=None), FeatureWeight(feature='x138', weight=-7.389568491608856e-06, std=None, value=None), FeatureWeight(feature='x159', weight=-7.37287315190505e-06, std=None, value=None)], pos_remaining=155, neg_remaining=162), proba=None, score=None, weighted_spans=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None)

In [62]:
eli5.explain_weights(clf)

Explanation(estimator="LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,\n          intercept_scaling=1, max_iter=100, multi_class='warn',\n          n_jobs=None, penalty='l2', random_state=None, solver='warn',\n          tol=0.0001, verbose=0, warm_start=False)", description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='<BIAS>', weight=1.4240277090452387, std=None, value=None), FeatureWeight(feature='x59', weight=1.3209353451147106e-05, std=None, value=None), FeatureWeight(feature='x171', weight=1.2018029353688385e-05, std=None, value=None), FeatureWeight(feature='x163', weight=1.0379213795322507e-05, std=None, value=None), FeatureWeight(feature='x68', weight=9.755164540692238e-06, std=None, value=None), FeatureWeight(feature='x309', weight=8.578184833479593e-06, std=None, value=None), FeatureWeight(feature='x169', weight=8.450286163857808e-06, std=None, value=None), FeatureWeight(feature='x69', weight=7.646178361836793e-06, std=None, value=None)], neg=[FeatureWeight(feature='x335', weight=-0.13298975543376212, std=None, value=None), FeatureWeight(feature='x110', weight=-1.2059058862897677e-05, std=None, value=None), FeatureWeight(feature='x6', weight=-1.1628030125637024e-05, std=None, value=None), FeatureWeight(feature='x7', weight=-9.032649517305785e-06, std=None, value=None), FeatureWeight(feature='x114', weight=-8.940534703431879e-06, std=None, value=None), FeatureWeight(feature='x306', weight=-8.899179041408397e-06, std=None, value=None), FeatureWeight(feature='x139', weight=-8.63346796966824e-06, std=None, value=None), FeatureWeight(feature='x111', weight=-8.291516754871603e-06, std=None, value=None), FeatureWeight(feature='x60', weight=-8.191094252628394e-06, std=None, value=None), FeatureWeight(feature='x75', weight=-7.745825589766683e-06, std=None, value=None), FeatureWeight(feature='x138', weight=-7.389568491608856e-06, std=None, value=None), FeatureWeight(feature='x159', weight=-7.37287315190505e-06, std=None, value=None)], pos_remaining=155, neg_remaining=162), proba=None, score=None, weighted_spans=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None)

In [5]:
epoch=mne.read_epochs('D:\\USB Drive\\NewEEG-200s\\New\\20131216_1310_epo.fif')
copy_epoch = epoch.copy()

Reading D:\USB Drive\NewEEG-200s\New\20131216_1310_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1759 matching events found
Applying baseline correction (mode: mean)
1759 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [15]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG','STI 014']);

In [7]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1759, 31868) (1759, 124, 257)


In [8]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 0.038636363636363635
-0.005500451520187966


In [9]:
y_pred

array([ 8.7, 14.1,  9.7, 16.4, 11.3, 12.9, 14.9,  9.9, 13.3,  9.3,  8.9,
        9.2,  8.6, 11.5, 12.3,  9.8, 12.4, 11.5, 11.3,  8.8, 10. , 13. ,
       13.7, 14.5, 10.3, 14.8, 11. , 16.7, 17.6, 13.2, 10.9, 10.3,  6.9,
       11.3,  8.3, 11. , 14.4, 12.5, 14.5, 10.7, 13.7, 12.8, 13.7, 11.7,
       12.2,  8.1, 12.3, 14.7, 10.7, 11.8, 19.9, 13.8, 14.8, 11.6, 13.5,
        8. ,  8.8, 11.7, 11.9,  8.6, 14.2, 14. , 13.7, 10.2, 12.2, 13. ,
        8.5,  7.9, 13.2, 16.2, 13.4, 11.8, 15.7,  9.7,  9.9, 10. , 13.2,
       14.6, 10.9, 15.8, 11.5, 14.4, 10.6, 13.7,  9.2, 11.2, 15.4, 10.2,
       13.4, 10.5, 14.2, 11.7, 13.1,  7.5, 12.8, 12. , 12.6, 11.1, 10.2,
        7.6, 10. , 16.8, 14.7, 11.8,  9.2, 10.4,  7.5, 11.1, 12.1,  9.1,
       15. , 10.6, 13.1, 10.4, 11.2, 10. , 12. , 14.5, 13.6, 12.7, 13.3,
        7.6,  8.2, 12.6, 14.9, 14.3, 12.2, 10. , 12.2, 12.3, 14. , 14.3,
       14.2,  7.7,  9.4,  8.2, 17.4, 11.9, 14.5, 10. , 14.8, 10.7, 13.3,
       10.3,  9.4,  8.5, 10.7, 17.4, 13.1, 11.7,  8

In [10]:
y_test

array([13,  6, 23,  4,  7,  4,  5,  9, 23,  8, 21, 20,  3, 11, 18, 11,  5,
       18, 22, 14, 11, 11,  4, 11, 11, 24,  8, 22,  3,  4, 16,  9, 16, 13,
       19, 18, 13, 12, 12,  2,  6, 19,  4, 10, 17,  6,  5, 14, 22,  3, 10,
       24, 20, 12, 24, 11,  8,  6, 17, 10, 13,  5, 20, 10, 10, 15,  3, 10,
       14, 11, 19,  9, 21,  7, 21, 21,  5, 20, 18,  9,  6, 23,  9,  7, 11,
       12,  8, 19, 22, 18, 10,  6, 17, 13, 21, 11, 19, 19, 19,  4,  4, 23,
       21,  7, 20, 24, 15, 21, 21, 19,  2,  2,  9,  3, 12, 24, 19, 21, 19,
        3, 21, 19,  8, 18, 22, 14,  4,  7,  8, 13, 24, 21, 24,  9, 16,  2,
       20,  9, 13, 21, 20,  4, 12, 18,  4,  9,  5, 11, 19, 19, 14, 21, 23,
       22, 10, 21, 18, 18, 11, 14, 22, 11,  4,  3, 22, 20,  5, 13, 23, 12,
        5, 24,  4,  6,  9, 13, 12, 22, 21,  4, 13, 21,  9,  6, 17, 19,  9,
        4,  6, 24, 13, 13, 16,  5, 24,  3, 15, 13,  6,  1, 14,  2,  7, 21,
       22, 22, 11, 20, 10, 11,  8, 11, 13, 24, 22, 12, 20,  4, 10, 13, 12,
       10, 18, 17,  6,  7

In [11]:
## Methods for convert our events into binary, 4: Language, 5:Non Language
def filter_event_id(epoch):
    newepoch=epoch
    epochs=[[]]
    j=0
    for i in range(len(newepoch.events[:,-1])):
        if i % 3 ==0:
            epochs.append([])
            j=j+1
            epochs[j].append(epoch.events[i,-1])
        else:
            epochs[j].append(epoch.events[i,-1])
    for events in epochs:
        if  4 in events or 1 in events or 13 in events or 16 in events:
            for i in range(len(events)):
                    events[i]=1
        else:
            for i in range(len(events)):
                    events[i]=0
    epochs.pop(0)
    flattened_list = [y for x in epochs for y in x]
    for i in range(len(newepoch.events)):
        newepoch.events[i]=flattened_list[i]
    return newepoch

In [12]:
epoch=filter_event_id(copy_epoch)
print(epoch.events[:,-1])

[1 1 1 ... 1 1 1]


In [64]:
## Vecorized to convert 3d to 2d
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1759, 31868) (1759, 124, 257)


In [40]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 0.5909090909090909
-0.028972900635664267


In [23]:
channel=[]
index=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances)):
    if importances[i]>0.00000001:
        channel.append(importances[i])
        index.append(i)
channel1=np.array(channel)
channel1.shape

(694,)

In [21]:
importances[0:300]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [24]:
channel=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances),257):
    channel.append(np.sum(importances[i:i+257]))
    channel1=np.array(channel)
    channel1.shape
channel1.shape

(124,)

In [25]:
import pandas as pd
feature_imp = pd.Series(channel1,index=epoch.ch_names).sort_values(ascending=False)
feature_imp

LB3     0.042611
R2      0.039519
RR11    0.026797
LD4     0.026451
LC4     0.024865
LC1     0.020261
R1      0.020083
LC3     0.017259
RR1     0.016639
RE4     0.015030
Lm      0.014740
L1      0.014542
LB4     0.013438
LL6     0.013171
L14     0.012953
LD2     0.012904
Z2      0.012670
LD5     0.012291
RB4     0.012008
Z13     0.011938
R14     0.011887
Z10     0.011666
Z14     0.011549
LL2     0.011412
RE3     0.011069
RR13    0.010723
RD3     0.010399
LL13    0.010128
LB1     0.010109
Z12     0.010095
          ...   
Z8      0.004451
R10     0.004440
R13     0.004398
LA1     0.004359
RD5     0.003634
Z5      0.003530
RA3     0.003499
LA4     0.003430
L6      0.003287
RR3     0.003278
L10     0.003101
L8      0.002901
LL12    0.002866
LE4     0.002732
LL9     0.002618
LB5     0.002562
Z4      0.002550
RR8     0.002469
R5      0.002428
RB5     0.002363
RR4     0.002281
L4      0.001930
RA2     0.001674
RR5     0.001500
RD1     0.001347
RR10    0.001288
LL3     0.000854
RA1     0.0000

In [45]:
channel=[]
index=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances)):
    if importances[i]>0.001:
        channel.append(importances[i])
        index.append(i)

channel1=np.array(channel)
index

[130,
 150,
 193,
 219,
 230,
 247,
 284,
 534,
 560,
 789,
 821,
 905,
 913,
 941,
 977,
 1029,
 1042,
 1066,
 1079,
 1092,
 1290,
 1314,
 1404,
 1450,
 1527,
 1627,
 1710,
 1861,
 1890,
 1910,
 1972,
 2095,
 2115,
 2153,
 2186,
 2237,
 2373,
 2488,
 2520,
 2611,
 2790,
 2851,
 2935,
 3012,
 3140,
 3255,
 3291,
 3394,
 3496,
 3585,
 3602,
 3713,
 3734,
 3740,
 3827,
 4096,
 4099,
 4226,
 4502,
 4542,
 4607,
 4653,
 4768,
 4846,
 4893,
 4976,
 5291,
 5522,
 5558,
 5649,
 5681,
 5782,
 5787,
 5863,
 6152,
 6154,
 6156,
 6230,
 6314,
 6528,
 6635,
 6662,
 6743,
 6872,
 6889,
 6982,
 6994,
 7007,
 7018,
 7021,
 7072,
 7112,
 7188,
 7392,
 7457,
 7625,
 7683,
 8014,
 8077,
 8147,
 8174,
 8203,
 8546,
 8556,
 8689,
 8713,
 8744,
 8809,
 8862,
 8872,
 9029,
 9108,
 9142,
 9187,
 9239,
 9255,
 9459,
 9573,
 9657,
 9925,
 10181,
 10612,
 10622,
 10649,
 10679,
 11008,
 11102,
 11215,
 11228,
 11230,
 11237,
 11258,
 11656,
 11679,
 11714,
 11762,
 11920,
 11934,
 11980,
 12221,
 12223,
 12404,

In [46]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
newdata=[]
j=0
for point in X:
    data=[]
    j=j+1
    print(j)
    for i in range(len(point)):
        if i in index:
            data.append(point[i])
    newdata.append(data)      

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [47]:
newdata=np.array(newdata)
newdata.shape
rs.get_n_splits(X)

(1759, 387)

In [70]:
X=newdata
y = epoch.events[:,-1]
from sklearn import metrics
clf = make_pipeline(Vectorizer(),
                   StandardScaler(),
                         LogisticRegression(solver='lbfgs',max_iter = 1000))

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 0.5159090909090909
0.5159090909090909


In [71]:
y_pred

array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,

In [56]:
y_test

array([1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,

In [62]:
(channel1)

0.8376819208242228

In [63]:
newdata

array([[-4.37699841e-06,  9.30988977e-06, -2.33632945e-06, ...,
         2.67602419e-06,  1.40388775e-06, -3.54825353e-06],
       [-1.57131373e-06, -5.60458659e-06, -1.28834905e-05, ...,
         6.80200051e-06,  5.66905259e-06, -2.29947730e-07],
       [-3.04635476e-06, -1.42238944e-07,  8.89745424e-06, ...,
         3.75282597e-06, -2.37985730e-06,  1.30728233e-06],
       ...,
       [-3.66457271e-06, -6.55752657e-06,  4.80728291e-06, ...,
         8.99816451e-07, -5.03623484e-07,  2.39356613e-06],
       [-2.81374835e-06,  1.52088832e-06,  3.31645774e-06, ...,
         2.51564764e-06,  5.30884551e-06, -1.59759020e-06],
       [ 6.85492037e-07, -2.91296791e-06,  7.22228859e-06, ...,
         1.25855994e-06, -6.56879543e-08,  9.22889528e-07]])

In [101]:

rs = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)
rs.get_n_splits(X)

5

In [104]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
X = newdata
Y = y = epoch.events[:,-1]

 
rf = RandomForestRegressor(n_estimators =10, random_state = 42)
scores = defaultdict(list)
j=0 

for train_idx, test_idx in rs.split(X):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
#crossvalidate the scores on a number of different random splits of the data

    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[j].append((acc-shuff_acc)/acc)
        j=j+1
print ("Features sorted by their score:")

Features sorted by their score:


In [103]:
X.shape[1]

387

In [105]:
sort=sorted([(round(np.mean(score), 4), feat) for feat, score in scores.items()], reverse=True)

In [110]:
len(sort)

1935

In [97]:
print(j)

0


In [98]:
rs = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)

In [99]:
rs.get_n_splits(X)

5

In [111]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1759, 31868) (1759, 124, 257)


In [112]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =110, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred()))
score = clf.score(X_test, y_test)

Accuracy: 0.6863636363636364
0.11128836035159227


In [114]:
import eli5
eli5.explain_weights(clf)

Weight,Feature
0.0026 ± 0.0194,x30871
0.0025 ± 0.0193,x28283
0.0024 ± 0.0184,x5718
0.0022 ± 0.0183,x18054
0.0021 ± 0.0180,x30991
0.0021 ± 0.0179,x28513
0.0019 ± 0.0158,x17292
0.0018 ± 0.0163,x28373
0.0016 ± 0.0146,x18071
0.0015 ± 0.0149,x30844


In [124]:
channel=[]
index=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances)):
    if importances[i]>0.0001:
        channel.append(importances[i])
        index.append(i)

channel1=np.array(channel)
channel1.shape
index[3000]

28175

In [125]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
newdata=[]
j=0
for point in X:
    data=[]
    print(++j)
    for i in range(len(point)):
        if i in index:
            data.append(point[i])
    newdata.append(data)       

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [126]:
newdata=np.array(newdata)
newdata.shape

(1759, 3563)

In [129]:
X=newdata
y = epoch.events[:,-1]
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =150, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.6818181818181818


In [132]:
X=newdata
y = epoch.events[:,-1]
from sklearn import metrics
clf = make_pipeline(Vectorizer(),
                   StandardScaler(),
                    RandomForestRegressor(n_estimators =150, random_state = 42))

    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.6772727272727272


array([0.60909091, 0.56363636, 0.44545455, 0.46363636, 0.35454545,
       0.57272727, 0.41818182, 0.5       , 0.42727273, 0.37272727,
       0.28181818, 0.30909091, 0.51818182, 0.49090909, 0.47272727,
       0.46363636, 0.46363636, 0.66363636, 0.37272727, 0.52727273,
       0.4       , 0.33636364, 0.50909091, 0.41818182, 0.33636364,
       0.49090909, 0.41818182, 0.5       , 0.61818182, 0.4       ,
       0.69090909, 0.4       , 0.67272727, 0.5       , 0.41818182,
       0.6       , 0.21818182, 0.3       , 0.49090909, 0.56363636,
       0.48181818, 0.39090909, 0.44545455, 0.6       , 0.65454545,
       0.41818182, 0.56363636, 0.53636364, 0.43636364, 0.36363636,
       0.46363636, 0.35454545, 0.4       , 0.48181818, 0.63636364,
       0.61818182, 0.45454545, 0.34545455, 0.46363636, 0.5       ,
       0.45454545, 0.40909091, 0.31818182, 0.46363636, 0.38181818,
       0.4       , 0.55454545, 0.5       , 0.65454545, 0.50909091,
       0.5       , 0.43636364, 0.35454545, 0.36363636, 0.53636

In [133]:
index

[9,
 20,
 23,
 27,
 30,
 32,
 33,
 35,
 37,
 38,
 46,
 47,
 51,
 52,
 65,
 68,
 81,
 95,
 118,
 130,
 134,
 138,
 144,
 145,
 150,
 159,
 160,
 161,
 165,
 182,
 185,
 189,
 193,
 201,
 212,
 219,
 222,
 223,
 225,
 230,
 233,
 234,
 247,
 251,
 263,
 284,
 299,
 346,
 348,
 355,
 365,
 367,
 372,
 375,
 390,
 403,
 418,
 428,
 434,
 442,
 447,
 466,
 491,
 494,
 533,
 534,
 535,
 536,
 560,
 561,
 585,
 602,
 614,
 620,
 623,
 627,
 630,
 632,
 634,
 647,
 650,
 651,
 666,
 673,
 700,
 702,
 706,
 707,
 719,
 753,
 759,
 764,
 767,
 768,
 775,
 787,
 789,
 793,
 807,
 821,
 822,
 829,
 853,
 855,
 857,
 861,
 862,
 866,
 871,
 875,
 905,
 919,
 922,
 923,
 933,
 940,
 941,
 945,
 947,
 949,
 956,
 969,
 976,
 977,
 982,
 984,
 991,
 995,
 1003,
 1012,
 1014,
 1024,
 1029,
 1034,
 1036,
 1038,
 1042,
 1046,
 1048,
 1054,
 1059,
 1066,
 1068,
 1076,
 1077,
 1079,
 1087,
 1090,
 1092,
 1096,
 1105,
 1107,
 1109,
 1121,
 1122,
 1127,
 1128,
 1129,
 1135,
 1136,
 1141,
 1142,
 1143,
 1150,

In [134]:
epoch=mne.read_epochs('D:\\USB Drive\\NewEEG-200s\\New\\20131216_1441_epo.fif')

Reading D:\USB Drive\NewEEG-200s\New\20131216_1441_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1753 matching events found
Applying baseline correction (mode: mean)
1753 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [135]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);

In [136]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1753, 32125) (1753, 125, 257)


In [137]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =10, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred()))
score = clf.score(X_test, y_test)
print(score)

Accuracy: 1.0
1.0


In [139]:
print(y_pred,y_test)


[ 5. 22. 22. 22. 20.  1. 21. 20.  7.  2. 21.  1. 22.  7. 13. 15.  1. 18.
  2.  5. 13. 18.  3. 18. 10.  1. 11.  1. 23. 15.  4. 20.  3. 17. 17.  3.
  7. 14. 24. 22.  4. 13. 15. 13.  1.  1. 10. 24.  2. 17. 17. 11. 14.  1.
 15.  3. 10. 20. 10.  5.  7. 16.  3. 12.  7.  2.  4.  9. 11. 18. 17. 24.
  2. 12. 19. 14.  4.  9.  9. 19. 14.  9. 13.  2.  5.  8.  1.  8. 12.  5.
 19. 22.  9. 20.  4.  8.  3.  9. 17. 10. 19.  1.  1. 22. 15. 20.  2. 15.
  1. 17. 16. 16.  9. 22. 18. 15.  6. 18.  6. 17.  7. 22. 14.  3. 15. 18.
  7. 22. 11.  9.  1. 19.  5.  4.  9.  8. 14. 22. 20. 21. 14.  8.  1. 21.
 23. 22. 18.  9.  6. 17. 20. 21. 12. 13. 12. 11. 24. 13. 15.  8.  3.  9.
 15.  9.  3. 14. 13.  7.  1. 24.  7.  1. 15. 17.  3. 10. 22. 17. 11.  7.
  7.  5. 18. 22. 18.  2. 12. 15. 22.  1. 10.  7. 21. 17. 15.  7.  5. 13.
  2.  6.  4. 19.  5. 16.  2. 11.  7. 19.  8.  2. 24. 18. 21.  9. 23.  2.
  1.  3. 12. 22. 19. 21.  2.  9. 11.  5. 16. 21. 10. 21. 15.  9.  8. 23.
 17. 21. 12.  2.  6. 23. 16.  4.  1.  6.  7.  6. 20

In [140]:
epoch.drop_channels(['STI 014'])

<EpochsFIF  |   1753 events (all good), 0 - 0.5 sec, baseline [0, 0], ~426.5 MB, data loaded,
 'b-f1-pic': 91
 'b-f1-snd': 74
 'b-f1-spk': 56
 'b-f1-wrd': 97
 'b-f2-pic': 91
 'b-f2-snd': 74
 'b-f2-spk': 57
 'b-f2-wrd': 97
 'b-tg-pic': 111
 'b-tg-snd': 55
 'b-tg-spk': 58
 'b-tg-wrd': 94
 'f-f1-pic': 84
 'f-f1-snd': 57
 'f-f1-spk': 70
 'f-f1-wrd': 55
 'f-f2-pic': 84
 'f-f2-snd': 57
 'f-f2-spk': 70
 'f-f2-wrd': 55
 'f-tg-pic': 54
 'f-tg-snd': 54
 'f-tg-spk': 72
 'f-tg-wrd': 86>

In [141]:
epoch=filter_event_id(epoch)
print(epoch.events[:,-1])

[0 0 0 ... 1 1 1]


In [142]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
print(X.shape,epoch.get_data().shape)

(1753, 31868) (1753, 124, 257)


In [143]:
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =110, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.5421412300683371


In [145]:
 channel=[]
index=[]
importances = np.array(clf.feature_importances_)
for i in range(0,len(importances)):
    if importances[i]>0.0001:
        channel.append(importances[i])
        index.append(i)

channel1=np.array(channel)
channel1.shape

(3683,)

In [146]:
index

[5,
 6,
 7,
 8,
 9,
 12,
 16,
 18,
 19,
 21,
 22,
 23,
 24,
 26,
 29,
 30,
 31,
 35,
 36,
 41,
 45,
 46,
 48,
 51,
 52,
 53,
 55,
 56,
 59,
 60,
 61,
 62,
 63,
 64,
 70,
 71,
 72,
 74,
 79,
 82,
 83,
 85,
 86,
 87,
 89,
 90,
 91,
 92,
 95,
 96,
 97,
 99,
 102,
 105,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 116,
 117,
 122,
 132,
 134,
 143,
 148,
 151,
 156,
 162,
 163,
 164,
 166,
 168,
 169,
 175,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 190,
 191,
 192,
 193,
 194,
 195,
 198,
 202,
 205,
 207,
 208,
 210,
 213,
 214,
 215,
 217,
 218,
 219,
 220,
 221,
 222,
 225,
 226,
 227,
 228,
 230,
 233,
 234,
 235,
 236,
 238,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 251,
 266,
 269,
 272,
 293,
 294,
 300,
 303,
 309,
 315,
 321,
 325,
 327,
 331,
 337,
 360,
 378,
 383,
 384,
 385,
 388,
 389,
 392,
 395,
 407,
 409,
 418,
 435,
 438,
 440,
 441,
 453,
 455,
 459,
 482,
 486,
 503,
 509,
 515,
 516,
 517,
 520,
 523,
 525,
 530,
 535,
 537,
 538,
 540,
 555,
 

In [147]:
X = epoch.get_data()
y = epoch.events[:,-1]
vec=Vectorizer()
X=vec.fit_transform(X)
newdata=[]
j=0
for point in X:
    data=[]
    j=j+1
    print(j)
    for i in range(len(point)):
        if i in index:
            data.append(point[i])
    newdata.append(data)    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [148]:
X=newdata
y = epoch.events[:,-1]
from sklearn import metrics
clf = RandomForestRegressor(n_estimators =110, random_state = 42)
    
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state = 42)
    
clf.fit(X_train, y_train)
    
y_pred=clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred.round()))

Accuracy: 0.55125284738041
